In [ ]:
import os
from pathlib import Path
import sys
from datetime import datetime

import pandas as pd
import polars as pl
import numpy as np
import joblib

# import kaggle_evaluation.nfl_inference_server
# Use robust importer to handle missing module in runtime
try:
    import kaggle_evaluation.nfl_inference_server as nfl_inf
except ModuleNotFoundError:
    from pathlib import Path
    root = Path('/kaggle/input')
    comp = None
    if root.exists():
        for p in root.iterdir():
            if p.is_dir() and 'nfl-big-data-bowl-2026-prediction' in p.name:
                comp = p
                break
    candidates = []
    if comp:
        candidates.append(comp / 'kaggle_evaluation')
    for p in root.iterdir() if root.exists() else []:
        if p.is_dir():
            candidates.append(p / 'kaggle_evaluation')
    for c in candidates:
        if c.exists():
            sys.path.insert(0, str(c.parent))
    import kaggle_evaluation.nfl_inference_server as nfl_inf

# Submission tracking info
SUBMISSION_CREATED = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
SUBMISSION_ID = datetime.now().strftime("%Y%m%d_%H%M%S")

# Global cache to load model/features once on first call
_MODEL_CACHE = {
    'loaded': False,
    'mx': None,
    'my': None,
    'feat_cols': None,
    'player_pos_vals': None,
    'attached_root': None,
    'model_path': None,
    'model_version': None,
    'submission_created': SUBMISSION_CREATED,
    'submission_id': SUBMISSION_ID
}


def _find_features_module(model_root=None):
    """Search all attached datasets for features.py
    
    Args:
        model_root: Optional path where model was found, to search there first
    """
    root = Path('/kaggle/input')
    if not root.exists():
        print("⚠️  /kaggle/input does not exist")
        return None
    
    print(f"\nSearching for features.py in {root}")
    
    # First check if features.py is in the same location as the model
    if model_root:
        print(f"  Checking model location: {model_root.name}")
        for candidate in [
            model_root / 'features.py',
            model_root / 'scikitlearn' / 'default' / '1' / 'features.py',
            model_root / 'for_kaggle' / 'features.py'
        ]:
            if candidate.exists():
                print(f"✓ Found features.py with model at: {candidate}")
                return candidate
    
    # Search all folders for features.py
    for p in root.iterdir():
        if p.is_dir() and 'nfl-big-data-bowl-2026-prediction' not in p.name:
            if p == model_root:
                continue  # Already checked above
            print(f"  Checking: {p.name}")
            # Check root level
            if (p / 'features.py').exists():
                print(f"✓ Found features.py in: {p}")
                return p / 'features.py'
            # Check nested locations
            for nested in p.rglob('features.py'):
                print(f"✓ Found features.py at: {nested}")
                return nested
    
    print("⚠️  No features.py found in any dataset")
    return None


def _find_model_file():
    """Search all attached datasets for model files
    
    Priority order:
    1. NEW pattern ONLY: nfl_model_v{timestamp}.pkl - newest first (defeats Kaggle caching!)
       - OLD pattern (best_model_*.pkl) is REJECTED to avoid NumPy compatibility issues
    """
    root = Path('/kaggle/input')
    if not root.exists():
        print("⚠️  /kaggle/input does not exist")
        return None, None
    
    print(f"\n=== Searching for Model ===")
    print(f"Available folders: {[p.name for p in root.iterdir() if p.is_dir()]}")
    print(f"POLICY: Only accepting nfl_model_v*.pkl (NEW pattern)")
    print(f"        Rejecting best_model_*.pkl (OLD pattern with NumPy issues)")
    
    candidates = sorted([p for p in root.iterdir() if p.is_dir()], key=lambda p: p.name)
    
    for p in candidates:
        # Skip the competition data folder
        if 'nfl-big-data-bowl-2026-prediction' in p.name:
            print(f"  Skipping competition folder: {p.name}")
            continue
        
        print(f"  Checking dataset: {p.name}")
        
        # Search for all .pkl files in standard locations
        search_patterns = [
            p / 'scikitlearn' / 'default' / '1',  # Model Registry
            p / 'models',  # Dataset format
            p / 'for_kaggle' / 'models',  # Nested format
        ]
        
        for search_dir in search_patterns:
            if not search_dir.exists():
                continue
            
            # Find model files - ONLY accept NEW pattern
            model_files = []
            
            # NEW pattern ONLY: nfl_model_v{timestamp}.pkl (to defeat Kaggle caching)
            new_pattern = sorted(search_dir.glob('nfl_model_v[0-9]*.pkl'), reverse=True)
            model_files.extend(new_pattern)
            
            # REJECT OLD pattern: best_model_*.pkl (causes NumPy compatibility issues)
            old_pattern = sorted(search_dir.glob('best_model_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_[0-9][0-9][0-9][0-9][0-9][0-9].pkl'), reverse=True)
            if old_pattern:
                print(f"    ⚠️  WARNING: Found OLD pattern models, REJECTING them:")
                for old_file in old_pattern:
                    print(f"      - {old_file.name} (causes PCG64 NumPy error)")
            
            # Fallback: REJECT non-versioned best_model.pkl
            if (search_dir / 'best_model.pkl').exists():
                print(f"    ⚠️  WARNING: Found best_model.pkl (non-versioned), REJECTING it")
            
            if model_files:
                chosen = model_files[0]
                print(f"✓ Found model in: {p}")
                print(f"  Model at: {chosen}")
                if len(model_files) > 1:
                    print(f"  (Found {len(model_files)} models, using newest: {chosen.name})")
                print(f"  🎯 Using NEW naming pattern (defeats caching, avoids NumPy issues)")
                return p, chosen
        
        # Fallback: search for any NEW pattern model recursively, REJECT old patterns
        all_new = list(p.glob('**/nfl_model_v*.pkl'))
        if all_new:
            chosen = sorted(all_new, reverse=True)[0]
            print(f"✓ Found model in: {p}")
            print(f"  Model at: {chosen}")
            return p, chosen
        
        # Check for old pattern and warn
        all_old = list(p.glob('**/best_model*.pkl'))
        if all_old:
            print(f"  ⚠️  Found OLD pattern models in {p.name} but REJECTING them:")
            for old_file in sorted(all_old, reverse=True)[:3]:
                print(f"    - {old_file.name}")
            print(f"     (These cause PCG64 BitGenerator NumPy compatibility errors)")
        
        print(f"  No valid NEW pattern model found in {p.name}")
    
    print("\n❌ ERROR: No valid model found (nfl_model_v*.pkl pattern required)")
    print("   Available but REJECTED old pattern models cause PCG64 NumPy errors")
    return None, None


def _lazy_load_model_and_modules():
    if _MODEL_CACHE['loaded']:
        return
    
    print("\n" + "=" * 60)
    print("LOADING MODEL AND FEATURES")
    print("=" * 60)
    print(f"Submission Created: {_MODEL_CACHE['submission_created']}")
    print(f"Submission ID: {_MODEL_CACHE['submission_id']}")
    
    # Find model file
    attached, model_path = _find_model_file()
    if not attached or not model_path:
        raise FileNotFoundError(
            "❌ No valid model found (nfl_model_v*.pkl pattern required).\n"
            "The old best_model_*.pkl files are rejected due to NumPy PCG64 compatibility issues.\n\n"
            "To fix:\n"
            "1. Delete old 'nfl-model-v*' datasets from Kaggle (without time component)\n"
            "2. Re-upload using timestamped dataset name: nfl-model-v{YYYYMMDD-HHMMSS}\n"
            "3. Ensure for_kaggle.zip contains nfl_model_v*.pkl (not best_model_*.pkl)\n"
            "4. Re-run the notebook"
        )
    
    _MODEL_CACHE['attached_root'] = attached
    _MODEL_CACHE['model_path'] = model_path
    
    # Find features.py (may be in different dataset than model, or with the model)
    features_path = _find_features_module(model_root=attached)
    if not features_path:
        raise FileNotFoundError(
            "\nfeatures.py not found in any attached dataset.\n"
            "Solutions:\n"
            "  1. Upload features.py as a separate dataset and attach it, OR\n"
            "  2. Upload for_kaggle.zip as a dataset (not Model Registry) - it contains both model and features.py"
        )
    
    # Add features location to path
    sys.path.insert(0, str(features_path.parent))
    print(f"\n✓ Loading features from: {features_path}")
    
    from features import add_time_lag_features, prepare_features, transform_for_inference  # noqa: F401
    _MODEL_CACHE['add_time_lag_features'] = add_time_lag_features
    _MODEL_CACHE['prepare_features'] = prepare_features
    _MODEL_CACHE['transform_for_inference'] = transform_for_inference
    
    # Store model path instead of loading model objects
    # This avoids gRPC serialization issues with sklearn models
    print(f"✓ Found model at: {model_path}")
    _MODEL_CACHE['model_path_str'] = str(model_path)
    
    # Load just to verify and print info
    meta = joblib.load(model_path)
    _MODEL_CACHE['feat_cols'] = meta['feature_columns']
    _MODEL_CACHE['player_pos_vals'] = meta.get('player_position_values', None)
    _MODEL_CACHE['loaded'] = True
    print(f"✓ Model verified with {len(_MODEL_CACHE['feat_cols'])} features")
    print(f"✓ X model random_state: {meta['models']['x'].random_state}")
    print(f"✓ Y model random_state: {meta['models']['y'].random_state}")
    
    # Extract model version from filename if available
    model_filename = model_path.name
    if 'nfl_model_v' in model_filename:
        version_id = model_filename.replace('nfl_model_v', '').replace('.pkl', '')
        _MODEL_CACHE['model_version'] = version_id
        print(f"✓ Model Version: {version_id}")
    
    print("=" * 60 + "\n")


def _to_pandas(df):
    if isinstance(df, pl.DataFrame):
        return df.to_pandas()
    return df


class NFLPredictor:
    """
    Callable class that loads model once on initialization.
    This avoids gRPC serialization issues - the model is loaded in the server process,
    not passed through the gRPC boundary.
    """
    def __init__(self):
        """Load model and features once during initialization."""
        print("\n" + "=" * 60)
        print("INITIALIZING PREDICTOR IN SERVER PROCESS")
        print("=" * 60)
        
        # Find and load model
        model_path = _find_model()
        if not model_path:
            raise FileNotFoundError("No valid model found")
        
        # Find and load features module
        features_path = _find_features_module(model_root=model_path.parent.parent)
        if not features_path:
            raise FileNotFoundError("features.py not found")
        
        sys.path.insert(0, str(features_path.parent))
        from features import add_time_lag_features, prepare_features, transform_for_inference
        
        print(f"✓ Loading model from: {model_path}")
        meta = joblib.load(model_path)
        
        # Store as instance variables
        self.mx = meta['models']['x']
        self.my = meta['models']['y']
        self.feat_cols = meta['feature_columns']
        self.player_pos_vals = meta.get('player_position_values', None)
        self.add_time_lag_features = add_time_lag_features
        self.prepare_features = prepare_features
        self.transform_for_inference = transform_for_inference
        
        print(f"✓ Model loaded successfully with {len(self.feat_cols)} features")
        print(f"✓ X model random_state: {self.mx.random_state}")
        print(f"✓ Y model random_state: {self.my.random_state}")
        print("=" * 60 + "\n")
    
    def __call__(self, test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
        """
        Predict function called by the NFL evaluation gateway.
        Returns Polars or Pandas DataFrame with columns ['x','y'].
        """
        # Convert inputs to pandas for feature pipeline
        test_pd = _to_pandas(test)
        test_in_pd = _to_pandas(test_input)

        # Merge like training: left join on identifiers if available
        key_cols = [c for c in ['game_id','play_id','nfl_id','frame_id'] 
                   if c in test_pd.columns and c in test_in_pd.columns]
        if key_cols:
            df = pd.merge(test_pd, test_in_pd, on=key_cols, how='left', suffixes=(None,'_in'))
        else:
            df = test_pd.copy()

        # Feature engineering for inference
        df = self.add_time_lag_features(df)
        _ = self.prepare_features(df)
        X_pred = self.transform_for_inference(df, self.feat_cols, self.player_pos_vals)

        # Predict
        px = self.mx.predict(X_pred)
        py = self.my.predict(X_pred)

        predictions = pd.DataFrame({'x': px, 'y': py})
        assert len(predictions) == len(test_pd)
        return predictions


# Create predictor instance - will be initialized once when server starts
predictor = NFLPredictor()

# Start inference server (serve on hidden test; local gateway otherwise)
inference_server = nfl_inf.NFLInferenceServer(predictor)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # Provide path to published public competition files for local gateway
    inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))